# Notebook 01 — Through-the-Cycle (TTC) PD (Give Me Some Credit)

**Overview**

- Objective: Build a defensible TTC-style PD workflow on the "Give Me Some Credit" dataset. This notebook is structured for Colab/Binder and follows the portfolio flow: Data load → EDA → Robust preprocessing (TTC-focused) → Model training → Evaluation → Save artefacts.

### Notes before running
- Expected dataset path: `data/raw/GiveMeSomeCredit/cs-training.csv`. Edit `DATA_PATH` below if necessary.
- This notebook focuses on robust preprocessing and avoids the NaN pitfalls encountered earlier. If you run into any column-specific NaNs, the preprocessing cell prints a breakdown so we can iterate fast.

### What this notebook contains (short)
1. Imports & data load (graceful if file missing)
2. Quick but informative EDA (head, types, missingness, simple univariate checks)
3. Robust preprocessing & feature engineering (imputation, outlier capping, AgeBucket handling, scaling). This is the updated cell you asked for.
4. Model pipeline (imputer + scaler + logistic regression) and evaluation (AUC, classification report)
5. Save a small model artifact to `artifacts/` if available

---

If anything breaks, **copy the exact traceback** and paste it here — I'll patch immediately.

In [ ]:
# --------------------------
# 1) Imports and load dataset
# --------------------------
import os
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix
import joblib

# Path to data (edit if your file is elsewhere)
DATA_PATH = 'data/raw/GiveMeSomeCredit/cs-training.csv'

print(f'Looking for dataset at: {DATA_PATH}')
if not os.path.exists(DATA_PATH):
    raise FileNotFoundError(f"Dataset not found at {DATA_PATH}. Please upload the file to that path or edit DATA_PATH.")

df = pd.read_csv(DATA_PATH)
print('\nLoaded dataset. Shape:', df.shape)
display(df.head())

In [ ]:
# --------------------------
# 2) Quick EDA (types, missingness, basic univariate checks)
# --------------------------
print('\nColumn data types and non-null counts:')
display(df.info())

print('\nMissing values per column:')
display(df.isnull().sum().sort_values(ascending=False))

print('\nBasic descriptive stats for numeric columns:')
display(df.describe().T)

# Target column check
target_col = 'SeriousDlqin2yrs'  # GiveMeSomeCredit target
if target_col not in df.columns:
    raise KeyError(f"Expected target column '{target_col}' not found in data. Columns: {list(df.columns)}")

print(f"\nTarget balance (value counts) for '{target_col}':")
display(df[target_col].value_counts(dropna=False))

# Quick univariate: default rate by simple age bins (demonstrative)
if 'age' in df.columns:
    display(df.groupby(pd.cut(df['age'], bins=[0,30,40,50,60,70,100]))[target_col].mean())

print('\nEDA complete — proceed to preprocessing cell below.')

In [ ]:
# =========================================================
# UPDATED Data Preparation & Feature Engineering (robust)
# - fixes post-imputation NaNs (pd.cut, derived cols, etc.)
# - prints detailed missing-col breakdown if assertion would fail
# =========================================================
df_prep = df.copy()

# --- 1) Impute numeric columns robustly ---
numeric_cols = df_prep.select_dtypes(include=[np.number]).columns.tolist()
if target_col in numeric_cols:
    numeric_cols.remove(target_col)

# Use median imputation (robust to outliers)
num_imputer = SimpleImputer(strategy='median')
df_prep[numeric_cols] = num_imputer.fit_transform(df_prep[numeric_cols])
missing_after_impute = int(df_prep[numeric_cols].isnull().sum().sum())
print(f'🔁 Missing values in numeric cols after median imputation: {missing_after_impute}')

# --- 2) Cap extreme outliers on important numeric features ---
for col in ['RevolvingUtilizationOfUnsecuredLines', 'DebtRatio']:
    if col in df_prep.columns:
        low = df_prep[col].quantile(0.01)
        high = df_prep[col].quantile(0.99)
        df_prep[col] = np.clip(df_prep[col], low, high)

# --- 3) Feature engineering (careful to avoid NaNs) ---
if 'age' in df_prep.columns:
    min_age = int(max(0, df_prep['age'].min() - 1))
    max_age = int(df_prep['age'].max() + 1)
    bins = [min_age, 30, 40, 50, 60, 70, 80, max_age]
    df_prep['AgeBucket'] = pd.cut(df_prep['age'], bins=bins, labels=False, include_lowest=True)
    df_prep['AgeBucket'] = df_prep['AgeBucket'].fillna(-1).astype(int)

if {'MonthlyIncome', 'DebtRatio'}.issubset(df_prep.columns):
    eps = 1e-8
    df_prep['IncomeToDebt'] = df_prep['MonthlyIncome'] / (df_prep['DebtRatio'] + eps)

# --- 4) Scaling selected numeric features (StandardScaler) ---
scaled_cols = [c for c in ['RevolvingUtilizationOfUnsecuredLines','DebtRatio','MonthlyIncome','IncomeToDebt'] if c in df_prep.columns]
if scaled_cols:
    scaler = StandardScaler()
    df_prep[scaled_cols] = scaler.fit_transform(df_prep[scaled_cols])

# --- 5) Final missingness check with helpful debug output ---
total_missing = int(df_prep.isnull().sum().sum())
print(f'🔎 Total missing values after all preprocessing steps: {total_missing}')
if total_missing == 0:
    print("✅ No missing values remain after preprocessing.")
else:
    print("❗ Preprocessing left missing values. Breakdown per column (top 50):")
    display(df_prep.isnull().sum().sort_values(ascending=False).head(50))
    raise AssertionError("⚠️ Missing values remain after preprocessing! Inspect above.")

print(f'📦 Prepared dataset shape: {df_prep.shape}')

In [ ]:
# --------------------------
# 4) Train/test split, modelling pipeline and evaluation
# --------------------------
FEATURE_COLS = [c for c in df_prep.columns if c != target_col]
X = df_prep[FEATURE_COLS]
y = df_prep[target_col].astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=42)
print('Train shape:', X_train.shape, 'Test shape:', X_test.shape)

# Build a safe pipeline (redundant imputer included to protect vs accidental NaNs)
pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
    ('clf', LogisticRegression(max_iter=500))
])

pipeline.fit(X_train, y_train)

y_pred_proba = pipeline.predict_proba(X_test)[:, 1]
y_pred = (y_pred_proba >= 0.5).astype(int)

auc = roc_auc_score(y_test, y_pred_proba)
print(f'ROC AUC on test set: {auc:.4f}')
print('\nClassification report:')
print(classification_report(y_test, y_pred, digits=4))
print('\nConfusion matrix:')
print(confusion_matrix(y_test, y_pred))

In [ ]:
# --------------------------
# 5) Save artefacts (model) and provide next steps
# --------------------------
ARTIFACT_DIR = 'artifacts/models'
Path(ARTIFACT_DIR).mkdir(parents=True, exist_ok=True)
MODEL_PATH = os.path.join(ARTIFACT_DIR, 'ttc_pd_logreg.joblib')
joblib.dump(pipeline, MODEL_PATH)
print(f'Model pipeline saved to: {MODEL_PATH}')

print('\nNext steps (suggested):')
print('- Expand EDA: IV/WOE, calibration plots, population stability (PSI), and vintage analysis.')
print('- Replace logistic with survival or hierarchical models if you have borrower-time series.')
print('- Implement cyclical adjustment: e.g., compute long-run average PD by grade and align intercepts.')